In [51]:
import pandas as pd

In [52]:
data=pd.read_csv(r"completedata.csv")


DROPPING EXTRA COLUMNS

In [53]:
data.columns

Index(['caseid', 'anestart', 'aneend', 'opstart', 'opend', 'adm', 'dis',
       'icu_days', 'death_inhosp', 'age', 'sex', 'height', 'weight', 'bmi',
       'asa', 'emop', 'department', 'optype', 'dx', 'opname', 'approach',
       'position', 'ane_type', 'preop_htn', 'preop_dm', 'preop_ecg',
       'preop_hb', 'preop_plt', 'preop_pt', 'preop_aptt', 'preop_cr', 'RISK'],
      dtype='object')

In [54]:
df=data[['age','bmi','asa','preop_htn','preop_dm','preop_ecg','preop_hb','preop_aptt','preop_cr','RISK']]
df

,age,bmi,asa,preop_htn,preop_dm,preop_ecg,preop_hb,preop_aptt,preop_cr,RISK
0,18,19.4,1,0,0,Normal Sinus Rhythm,14.7,34.1,0.67,Due Risk
1,18,16.6,1,0,0,Normal Sinus Rhythm,14.8,30.9,0.96,Low
2,18,24.5,1,0,0,Normal Sinus Rhythm,12.6,30.6,0.58,Due Risk
3,18,20.7,1,0,0,Normal Sinus Rhythm,16.4,29.4,0.92,Due Risk
4,19,20.7,1,0,0,Normal Sinus Rhythm,14.8,38.1,0.91,Due Risk
...,...,...,...,...,...,...,...,...,...,...
5479,37,16.8,6,0,0,Normal Sinus Rhythm,9.8,31.8,0.55,High
5480,37,16.8,6,0,0,Normal Sinus Rhythm,9.8,31.8,0.55,High
5481,64,22.2,6,0,0,Normal Sinus Rhythm,11.6,26.9,0.69,High
5482,69,25.6,6,1,0,Normal Sinus Rhythm,12.9,57.5,1.51,High


In [55]:
df.dtypes

age             int64
bmi           float64
asa             int64
preop_htn       int64
preop_dm        int64
preop_ecg      object
preop_hb      float64
preop_aptt    float64
preop_cr      float64
RISK           object
dtype: object

ENCODING ACCORDING TO MEDICAL RANGES

In [56]:
df.head(5)

,age,bmi,asa,preop_htn,preop_dm,preop_ecg,preop_hb,preop_aptt,preop_cr,RISK
0,18,19.4,1,0,0,Normal Sinus Rhythm,14.7,34.1,0.67,Due Risk
1,18,16.6,1,0,0,Normal Sinus Rhythm,14.8,30.9,0.96,Low
2,18,24.5,1,0,0,Normal Sinus Rhythm,12.6,30.6,0.58,Due Risk
3,18,20.7,1,0,0,Normal Sinus Rhythm,16.4,29.4,0.92,Due Risk
4,19,20.7,1,0,0,Normal Sinus Rhythm,14.8,38.1,0.91,Due Risk


In [57]:
def custom_preprocessing(df):
    agebins = [0,49,69,79,150]
    labels=[1,2,3,4]
    df['age'] = pd.cut(df['age'], bins=agebins, labels=labels, include_lowest=True,ordered=False)

    bmibins = [0,7.99,11.99,18.49,24.99,29.99,34.99,100]
    labels=[4,3,2,1,2,3,4]
    df['bmi'] = pd.cut(df['bmi'], bins=bmibins, labels=labels, include_lowest=True,ordered=False)

    crbins = [0,0.99,1.99,3.99,1000]
    labels=[1,2,3,4]
    df['preop_cr'] = pd.cut(df['preop_cr'], bins=crbins, labels=labels, include_lowest=True,ordered=False)
    
    apttbins = [0,39.99,1000]
    labels=[1,4]
    df['preop_aptt'] = pd.cut(df['preop_aptt'], bins=apttbins, labels=labels, include_lowest=True,ordered=False)
    
    hbbins = [0,7,9,11,100]
    labels=[4,3,2,1]
    df['preop_hb'] = pd.cut(df['preop_hb'], bins=hbbins, labels=labels, include_lowest=True,ordered=False)

    df['asa']=df['asa'].map({1:1,2:2,3:3,4:4,5:4,6:4})

    df['preop_htn']=df['preop_htn'].map({0:1, 1:4})

    df['preop_dm']=df['preop_dm'].map({0:1, 1:4})
    import numpy as np
    df['preop_ecg'] = np.where(df['preop_ecg'] == 'Normal Sinus Rhythm', 1, 3)

    df['age']=df['age'].astype("int64")
    df['bmi']=df['bmi'].astype("int64")
    df['preop_hb']=df['preop_hb'].astype("int64")
    df['preop_cr']=df['preop_cr'].astype("int64")
    df['preop_aptt']=df['preop_aptt'].astype("int64")
    
    return df
    

In [58]:
import pickle 
file = open('datapreprocessing.pickle','wb')
pickle.dump(custom_preprocessing, file)
file.close()


In [59]:
file = open("datapreprocessing.pickle", 'rb')
func = pickle.load(file)
file.close()

In [66]:
import cloudpickle
file = open('datapreprocessing.pickle','wb')
cloudpickle.dump(custom_preprocessing, file)
file.close()

In [60]:
df=func(df)
df.head(5)

C:\Users\ekasp\AppData\Local\Temp\ipykernel_18268\120112404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = pd.cut(df['age'], bins=agebins, labels=labels, include_lowest=True,ordered=False)
C:\Users\ekasp\AppData\Local\Temp\ipykernel_18268\120112404.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bmi'] = pd.cut(df['bmi'], bins=bmibins, labels=labels, include_lowest=True,ordered=False)
C:\Users\ekasp\AppData\Local\Temp\ipykernel_18268\120112404.py:12: SettingWithCopyWarning: 
A value 

,age,bmi,asa,preop_htn,preop_dm,preop_ecg,preop_hb,preop_aptt,preop_cr,RISK
0,1,1,1,1,1,1,1,1,1,Due Risk
1,1,2,1,1,1,1,1,1,1,Low
2,1,1,1,1,1,1,1,1,1,Due Risk
3,1,1,1,1,1,1,1,1,1,Due Risk
4,1,1,1,1,1,1,1,1,1,Due Risk


In [ ]:
'''agebins = [0,49,69,79,150]
labels=['Due Risk','Low','Moderate','High']
df['age'] = pd.cut(df['age'], bins=agebins, labels=labels, include_lowest=True,ordered=False)
df.head(5)
'''

In [ ]:
'''
bmibins = [0,7.99,11.99,18.49,24.99,29.99,34.99,100]
labels=['High','Moderate','Low','Due Risk','Low','Moderate','High']
df['bmi'] = pd.cut(df['bmi'], bins=bmibins, labels=labels, include_lowest=True,ordered=False)
'''

In [ ]:
''' 
crbins = [0,0.99,1.99,3.99,1000]
labels=['Due Risk','Low','Moderate','High']
df['preop_cr'] = pd.cut(df['preop_cr'], bins=crbins, labels=labels, include_lowest=True,ordered=False)
df.head(5)
'''

In [ ]:
''' 
apttbins = [0,39.99,1000]
labels=['Due Risk','High']
df['preop_aptt'] = pd.cut(df['preop_aptt'], bins=apttbins, labels=labels, include_lowest=True,ordered=False)
df.head(5)
'''

In [ ]:
''' 
hbbins = [0,7,9,11,100]
labels=['High','Moderate','Low','Due Risk']
df['preop_hb'] = pd.cut(df['preop_hb'], bins=hbbins, labels=labels, include_lowest=True,ordered=False)
df
'''

In [ ]:
'''
df['asa']=df['asa'].map({1: 'Due Risk', 2: 'Low',3: 'Moderate',4:'High',5:'High',6:'High'})
'''

In [ ]:
'''
df['preop_htn']=df['preop_htn'].map({0: 'Due Risk', 1:'High'})
'''

In [ ]:
''' 
df['preop_dm']=df['preop_dm'].map({0: 'Due Risk', 1:'High'})
'''

In [ ]:
''' 
import numpy as np
df['preop_ecg'] = np.where(df['preop_ecg'] == 'Normal Sinus Rhythm', 'Due Risk', 'Moderate')
'''

In [61]:
df

,age,bmi,asa,preop_htn,preop_dm,preop_ecg,preop_hb,preop_aptt,preop_cr,RISK
0,1,1,1,1,1,1,1,1,1,Due Risk
1,1,2,1,1,1,1,1,1,1,Low
2,1,1,1,1,1,1,1,1,1,Due Risk
3,1,1,1,1,1,1,1,1,1,Due Risk
4,1,1,1,1,1,1,1,1,1,Due Risk
...,...,...,...,...,...,...,...,...,...,...
5479,1,2,4,1,1,1,2,1,1,High
5480,1,2,4,1,1,1,2,1,1,High
5481,2,1,4,1,1,1,1,1,1,High
5482,2,2,4,4,1,1,1,4,2,High


In [62]:
df['RISK']=df['RISK'].map({'Due Risk':1,'Low':2,'Moderate':3,'High':4})

C:\Users\ekasp\AppData\Local\Temp\ipykernel_18268\123647692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RISK']=df['RISK'].map({'Due Risk':1,'Low':2,'Moderate':3,'High':4})


In [63]:
df

,age,bmi,asa,preop_htn,preop_dm,preop_ecg,preop_hb,preop_aptt,preop_cr,RISK
0,1,1,1,1,1,1,1,1,1,1
1,1,2,1,1,1,1,1,1,1,2
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5479,1,2,4,1,1,1,2,1,1,4
5480,1,2,4,1,1,1,2,1,1,4
5481,2,1,4,1,1,1,1,1,1,4
5482,2,2,4,4,1,1,1,4,2,4


In [64]:
df['RISK'].value_counts()

4    2373
3    1896
2     814
1     401
Name: RISK, dtype: int64

In [ ]:
'''
df['age']=df['age'].astype("int64")
df['bmi']=df['bmi'].astype("int64")
df['preop_hb']=df['preop_hb'].astype("int64")
df['preop_cr']=df['preop_cr'].astype("int64")
df['preop_aptt']=df['preop_aptt'].astype("int64")
'''

In [65]:
df.dtypes

age           int64
bmi           int64
asa           int64
preop_htn     int64
preop_dm      int64
preop_ecg     int32
preop_hb      int64
preop_aptt    int64
preop_cr      int64
RISK          int64
dtype: object

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import r2_score
from sklearn import metrics



In [ ]:
X=df.drop('RISK', axis=1)
y=df['RISK']

In [ ]:
y.value_counts()

In [ ]:
from pycaret.classification import *
s = setup(data=df, target='RISK')

cm = compare_models()


Without undersampling
#lr
#knn
#DT
#RF
#Gradient Boosting
#Extra Trees Classifier


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)

In [ ]:
lr = LogisticRegression(random_state =42,multi_class='auto',max_iter=10000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
acc_lr=accuracy_score(y_test,y_pred)
print(acc_lr)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn=accuracy_score(y_test,y_pred)
print(acc_knn)

In [ ]:
from sklearn.model_selection import GridSearchCV
tree_para = {'criterion':['gini','entropy'],'max_depth':[2,4,5,6,8,9,10,15,20,40,50,90,120,150],'min_samples_leaf':[2,4,5,6,8,9,10,15,20]}
dt = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)
dt.fit(X_train, y_train)
dt.best_params_


In [ ]:
y_pred = dt.predict(X_test)
acc_dt=accuracy_score(y_test,y_pred)
print(acc_dt)


In [ ]:
rf = RandomForestClassifier()

param_grid = {
    "n_estimators": [50, 100, 150,200, 500, 1000],
    "max_depth": [1,5, 10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2,3,4,5],
    "max_features": ["auto", "sqrt", "log2"]
}
rf = RandomizedSearchCV(rf, param_grid, cv=5)
rf.fit(X_train, y_train)
rf.best_params_

In [ ]:
y_pred = rf.predict(X_test)
acc_rf=accuracy_score(y_test,y_pred)
print(acc_rf)

In [ ]:
gradient = GradientBoostingClassifier()

param_grid = {
    "n_estimators": [50, 100, 200, 500],
    "max_depth": [1,5, 10, 20, 30, None],
    "learning_rate": [0.1,0.01,0.001,0.5,0.05],
}
gra = RandomizedSearchCV(gradient, param_grid, cv=10)
gra.fit(X_train, y_train)
gra.best_params_

In [ ]:
y_pred = gra.predict(X_test)
acc_gradient=accuracy_score(y_test,y_pred)
print(acc_gradient)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y1_train = le.fit_transform(y_train)

xgb_clf = xgb.XGBClassifier()
param_grid = {
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [3, 5, 7, 9],
    "learning_rate": [0.1, 0.3, 0.5,1.0],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 5],
    "reg_alpha": [0, 1, 5],
    "reg_lambda": [0, 1, 5]
}

xgbr = RandomizedSearchCV(xgb_clf, param_grid, n_iter=100, cv=5)
xgbr.fit(X_train, y1_train)
xgbr.best_params_

In [ ]:
y_pred = xgbr.predict(X_test)
y1_test = le.fit_transform(y_test)
acc_xgbr=accuracy_score(y1_test,y_pred)
print(acc_xgbr)

In [ ]:
import pickle

file = open('dtn.pickle','wb')
pickle.dump(dt, file)
file.close()

file = open('lrn.pickle','wb')
pickle.dump(lr, file)
file.close()

file = open('knnn.pickle','wb')
pickle.dump(knn, file)
file.close()

file = open('gran.pickle','wb')
pickle.dump(gra, file)
file.close()

file = open('rfn.pickle','wb')
pickle.dump(rf, file)
file.close()

file = open('xgbrn.pickle','wb')
pickle.dump(xgbr, file)
file.close()

With Undersampling 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler()
X1,y1 = undersample.fit_resample(X, y)
y1.value_counts()

In [ ]:
'''
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='not majority'))
X1, y1 = resample.fit_resample(X, y)
y.value_counts()
'''
X1,

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X1,y1,test_size=0.5)

In [ ]:
lr = LogisticRegression(random_state =42,multi_class='auto',max_iter=10000)
lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_test)
acc_lr=accuracy_score(y_test,y_pred1)
print(acc_lr)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred2 = knn.predict(X_test)
acc_knn=accuracy_score(y_test,y_pred2)
print(acc_knn)

In [ ]:
from sklearn.model_selection import GridSearchCV
tree_para = {'criterion':['gini','entropy'],'max_depth':[2,4,5,6,8,9,10,15,20,40,50,90,120,150],'min_samples_leaf':[2,4,5,6,8,9,10,15,20]}
dt = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)
dt.fit(X_train, y_train)
dt.best_params_

In [ ]:
dt.fit(X_train, y_train)
y_pred3 = dt.predict(X_test)
acc_dt=accuracy_score(y_test,y_pred3)
print(acc_dt)


In [ ]:
rf = RandomForestClassifier()

param_grid = {
    "n_estimators": [50, 100, 150,200, 500, 1000],
    "max_depth": [1,5, 10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2,3,4,5],
    "max_features": ["auto", "sqrt", "log2"]
}
rf = RandomizedSearchCV(rf, param_grid, cv=5)
rf.fit(X_train, y_train)
rf.best_params_

In [ ]:
y_pred4 = rf.predict(X_test)
acc_rf=accuracy_score(y_test,y_pred4)
print(acc_rf)

In [ ]:
gradient = GradientBoostingClassifier()

param_grid = {
    "n_estimators": [50, 100, 200, 500],
    "max_depth": [1,5, 10, 20, 30, None],
    "learning_rate": [0.1,0.01,0.001,0.5,0.05],
}
gradient = RandomizedSearchCV(gradient, param_grid, cv=10)
gradient.fit(X_train, y_train)
gradient.best_params_

In [ ]:
y_pred5 = gradient.predict(X_test)
acc_gradient=accuracy_score(y_test,y_pred5)
print(acc_gradient)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y1_train = le.fit_transform(y_train)

xgb_clf = xgb.XGBClassifier()
param_grid = {
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [3, 5, 7, 9],
    "learning_rate": [0.1, 0.3, 0.5,1.0],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 5],
    "reg_alpha": [0, 1, 5],
    "reg_lambda": [0, 1, 5]
}

xgbr = RandomizedSearchCV(xgb_clf, param_grid, n_iter=100, cv=5)
xgbr.fit(X_train, y1_train)
xgbr.best_params_

In [ ]:
y_pred6 = xgbr.predict(X_test)
y1_test = le.fit_transform(y_test)
acc_xgbr=accuracy_score(y1_test,y_pred6)
print(acc_xgbr)

In [ ]:
models = list()
models.append(('dt', dt))
models.append(('lr', lr))
models.append(('knn', knn ))
models.append(('gradient', gradient))
models.append(('rf', rf))
models.append(('xgbr',xgbr))

ensemble = VotingClassifier(estimators=models, voting='hard')
ensemble.fit(X_train, y_train)
yhat = ensemble.predict(X_test)
acc_ensemb=accuracy_score(y_test,yhat)
print(acc_ensemb)

In [ ]:
import pickle

file = open('dtn.pickle','wb')
pickle.dump(dt, file)
file.close()

file = open('lrn.pickle','wb')
pickle.dump(lr, file)
file.close()

file = open('knnn.pickle','wb')
pickle.dump(knn, file)
file.close()

file = open('gran.pickle','wb')
pickle.dump(gradient, file)
file.close()

file = open('rfn.pickle','wb')
pickle.dump(rf, file)
file.close()

file = open('xgbrn.pickle','wb')
pickle.dump(xgbr, file)
file.close()

file = open('ensemble.pickle','wb')
pickle.dump(ensemble, file)
file.close()

With validation on undersampled data


In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ['Model','Validation Accuracy','Test Accuracy','R2_score','ROC_AUC','Precision'])
df

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X1, y1, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
lr = LogisticRegression(random_state =42,multi_class='auto',max_iter=10000)
lr.fit(X_train, y_train)

y_pred_val = lr.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = lr.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = lr.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'Logistic Regression','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

y_pred_val = knn.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = knn.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = knn.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'KNN','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
from sklearn.model_selection import GridSearchCV
tree_para = {'criterion':['gini','entropy'],'max_depth':[2,4,5,6,8,9,10,15,20,40,50,90,120,150],'min_samples_leaf':[2,4,5,6,8,9,10,15,20]}
dt = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)
dt.fit(X_train, y_train)
dt.best_params_

In [ ]:
y_pred_val = dt.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = dt.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = dt.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'Decision Tree','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    "n_estimators": [50, 100, 150,200, 500, 1000],
    "max_depth": [1,5, 10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2,3,4,5],
    "max_features": ["auto", "sqrt", "log2"]
}
rf = RandomizedSearchCV(rf, param_grid, cv=5)
rf.fit(X_train, y_train)


In [ ]:
y_pred_val = rf.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = rf.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = rf.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'Random Forest','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
gradient = GradientBoostingClassifier()

param_grid = {
    "n_estimators": [50, 100, 200, 500],
    "max_depth": [1,5, 10, 20, 30, None],
    "learning_rate": [0.1,0.01,0.001,0.5,0.05],
}
gra = RandomizedSearchCV(gradient, param_grid, cv=10)
gra.fit(X_train, y_train)

In [ ]:
y_pred_val = gra.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = gra.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = gra.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'Gradient Boosting','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
df

In [ ]:
models = list()
models.append(('dt', dt))
models.append(('lr', lr))
models.append(('knn', knn ))
models.append(('gradient', gra))

ensemble = VotingClassifier(estimators=models, voting='soft')
ensemble.fit(X_train, y_train)


In [ ]:
y_pred_val = ensemble.predict(X_valid)
val_acc=accuracy_score(y_valid, y_pred_val)

y_pred_test = ensemble.predict(X_test)
test_acc=accuracy_score(y_test,y_pred_test)

r2 = r2_score(y_test,y_pred_test)

pred_prob = ensemble.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,pred_prob,multi_class="ovo")

pre=precision_score(y_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'Ensemble','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
le = LabelEncoder()
y1_train = le.fit_transform(y_train)

xgb_clf = xgb.XGBClassifier()
param_grid = {
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [3, 5, 7, 9],
    "learning_rate": [0.1, 0.3, 0.5,1.0],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 5],
    "reg_alpha": [0, 1, 5],
    "reg_lambda": [0, 1, 5]
}

xgbr = RandomizedSearchCV(xgb_clf, param_grid, n_iter=100, cv=5)
xgbr.fit(X_train, y1_train)
xgbr.best_params_

In [ ]:
y1_test = le.fit_transform(y_test)
y1_valid=le.fit_transform(y_valid)

In [ ]:
y_pred_val = xgbr.predict(X_valid)
val_acc=accuracy_score(y1_valid, y_pred_val)

y_pred_test = xgbr.predict(X_test)
test_acc=accuracy_score(y1_test,y_pred_test)

r2 = r2_score(y1_test,y_pred_test)

pred_prob = xgbr.predict_proba(X_test)
auc = metrics.roc_auc_score(y1_test,pred_prob,multi_class="ovo")

pre=precision_score(y1_test,y_pred_test, average='micro')

print("Validation accuracy: ",val_acc)
print("Test accuracy: ",test_acc)
print("Precision: ",pre)
print('r2 score :',r2)
print('Auc :',auc)


df = df.append({'Model' : 'xgbr','Validation Accuracy':val_acc,'Test Accuracy':test_acc,'R2_score':r2,'ROC_AUC':auc,'Precision' : pre},ignore_index = True)

In [ ]:
df=df.reset_index(drop=True)
df

In [ ]:
import pickle

file = open('dtv.pickle','wb')
pickle.dump(dt, file)
file.close()

file = open('lrv.pickle','wb')
pickle.dump(lr, file)
file.close()

file = open('knnv.pickle','wb')
pickle.dump(knn, file)
file.close()

file = open('grav.pickle','wb')
pickle.dump(gra, file)
file.close()

file = open('rfv.pickle','wb')
pickle.dump(rf, file)
file.close()

file = open('xgbrv.pickle','wb')
pickle.dump(xgbr, file)
file.close()

file = open('ensemble..pickle','wb')
pickle.dump(ensemble, file)
file.close()

In [ ]:
import pandas as pd
import  numpy as np
import math as m

dataset=df.drop('Model',axis=1)
weight=[0.8,1,0.6,0.6,0.6]
impacts=['+','+','+','+','+']

data=dataset.iloc[:,:].values.astype(float)
data


In [ ]:
(r,c)=data.shape

s=sum(weight)
for i in range(c):
    weight[i]/=s
SsumofC=[0]*(c)
#calculate sum of squares of all columns
for i in range(0,r):
    for j in range(0,c):
        SsumofC[j]=SsumofC[j]+(data[i][j]*data[i][j])

#calculate root of sum of squares of all columns
for j in range(c):
    SsumofC[j]=m.sqrt(SsumofC[j])
#Vector normalization and multiplication by column weight
for i in range(r):
    for j in range(c):
        data[i][j]/=SsumofC[j]
        data[i][j]*=weight[j]
## this becomes WEIGHTED NORMALIZED MATRIX

#we find ideak best and worst values. 

ideal_best=np.amax(data,axis=0) # MAX IN VERTICAL COL
ideal_worst=np.amin(data,axis=0) # MIN IN EACH COL
for i in range(len(impacts)):
    if(impacts[i]=='-'):         # SWAPPING TO STORE REQUIRED IN ideal_best
        temp=ideal_best[i]
        ideal_best[i]=ideal_worst[i]
        ideal_worst[i]=temp
#find euclidean distance with both best and worst ideal values
best_dist=list()
worst_dist=list()

for i in range(r):
    s=0
    for j in range(c):
        s+=pow((data[i][j]-ideal_best[j]), 2)

    best_dist.append(float(pow(s,0.5)))


for i in range(r):
    s=0
    for j in range(c):
        s+=pow((data[i][j]-ideal_worst[j]), 2)

    worst_dist.append(float(pow(s,0.5)))


score=dict()
#performance score is mapped in a dict .values are stored in a 
for i in range(r):
    score[i+1]=worst_dist[i]/(worst_dist[i]+best_dist[i])

a=list(score.values())
b=sorted(list(score.values()) , reverse=True)

rank=dict()
#rank starts from 1 so index+1
#we take element of a. find its index in sorted list b. and that index+1 is final rank
for i in range(len(a)):
    rank[(b.index(a[i]) + 1)] = a[i]
    b[b.index(a[i])] =-b[b.index(a[i])]


a=list(rank.values())
rr=list(rank.keys())

#forming the output file 
out={'name' : df['Model'],'score' : a , 'Rank':rr}
output=pd.DataFrame(out)
output



In [ ]:
yytest = gra.predict(X_temp)
accuracy=accuracy_score(y_temp,yytest)
print(accuracy)